In [2]:
import json
import os
from datasets import load_dataset, Dataset
import datetime
import time


class AdvancedRAGEvaluationGenerator:
    """
    Clase para generar respuestas y dataset RAGAS usando AdvancedRAGPipeline.
    Maneja indexación de documentos y procesamiento en lotes.
    """
    def __init__(self, rag_pipeline, clean_dataset, papers_dir=None, batch_size=3, delay=30):
        """
        Args:
            rag_pipeline: Instancia de AdvancedRAGPipeline.
            clean_dataset: Dataset limpio (qa_clean_dataset).
            papers_dir: Directorio con PDFs (None si no hay PDFs).
            batch_size: Tamaño del lote para procesamiento.
            delay: Segundos de espera entre lotes.
        """
        self.rag = rag_pipeline
        self.dataset = clean_dataset
        self.papers_dir = papers_dir
        self.batch_size = batch_size
        self.delay = delay
        self.results_file = "advanced_rag_evaluation_results.json"
        self.results = self.load_previous_results()
        print(f"✅ Inicializado AdvancedRAGEvaluationGenerator con {len(clean_dataset)} ejemplos")

    def load_previous_results(self):
        """Cargar resultados previos para reanudar."""
        if os.path.exists(self.results_file):
            with open(self.results_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return []

    def save_results(self):
        """Guardar resultados incrementalmente."""
        with open(self.results_file, 'w', encoding='utf-8') as f:
            json.dump(self.results, f, ensure_ascii=False, indent=2)

    def index_documents(self):
        """Indexar documentos referenciados por paper_id en Qdrant."""
        print("Indexando documentos...")
        papers_dataset = load_dataset("UKPLab/PeerQA", "papers")["test"]
        indexed, failed = self.rag.setup_peerqa_retriever(papers_dataset=papers_dataset)
        print(f"Indexación completada: {indexed} exitosos, {failed} fallidos")
        return indexed, failed

    def generate_responses_batch(self, model_type='openrouter', start_idx=None, max_samples=30):
        """
        Generar respuestas en lotes.
        
        Args:
            model_type: 'openrouter' o 'groq'.
            start_idx: Índice inicial (None para continuar desde resultados previos).
            max_samples: Máximo número de muestras a procesar.
        """
        total_samples = len(self.dataset)
        if start_idx is None:
            start_idx = len(self.results)

        end_idx = min(start_idx + max_samples, total_samples) if max_samples else total_samples

        print(f"Generando respuestas con {model_type}")
        print(f"Procesando desde índice {start_idx} hasta {end_idx}")
        print(f"Lotes de {self.batch_size} con {self.delay}s de espera")
        print(f"Ya procesados: {len(self.results)} ejemplos")

        for i in range(start_idx, end_idx, self.batch_size):
            batch_end = min(i + self.batch_size, end_idx)
            current_batch = range(i, batch_end)

            print(f"\n🔄 Procesando lote {i//self.batch_size + 1}: índices {i}-{batch_end-1}")

            for idx in current_batch:
                try:
                    example = self.dataset[idx]
                    question = example['question']
                    print(f"[{idx+1}/{end_idx}] Procesando pregunta: {question[:50]}...")

                    # Generar respuesta con AdvancedRAGPipeline
                    rag_result = self.rag.query_rag(question, model=model_type, top_k=5)

                    # Preparar resultado para RAGAS
                    result = {
                        'question': question,
                        'answer': rag_result['answer'],
                        'contexts': rag_result['contexts'],
                        'ground_truth': example['ground_truth'],
                        'paper_id': example['paper_id'],
                        'model_used': model_type,
                        'timestamp': datetime.now().isoformat(),
                        'processed_idx': idx
                    }
                    self.results.append(result)
                    print(f"  ✅ Completado!")

                except Exception as e:
                    print(f"Error en índice {idx}: {str(e)}")
                    error_result = {
                        'question': self.dataset[idx]['question'] if idx < len(self.dataset) else "Error loading",
                        'answer': f"ERROR: {str(e)}",
                        'contexts': [],
                        'ground_truth': self.dataset[idx].get('ground_truth', 'N/A') if idx < len(self.dataset) else "N/A",
                        'paper_id': self.dataset[idx].get('paper_id', 'N/A') if idx < len(self.dataset) else "N/A",
                        'model_used': model_type,
                        'timestamp': datetime.now().isoformat(),
                        'processed_idx': idx,
                        'error': True
                    }
                    self.results.append(error_result)

            # Guardar progreso después de cada lote
            self.save_results()
            print(f"💾 Progreso guardado: {len(self.results)} ejemplos completados")

            if batch_end < end_idx:
                print(f"⏳ Esperando {self.delay} segundos...")
                time.sleep(self.delay)

        print(f"\n🎉 ¡Proceso completado! Total: {len(self.results)} ejemplos")
        return self.results

    def get_ragas_dataset(self, filter_errors=True):
        """
        Crear dataset para RAGAS.
        
        Args:
            filter_errors: Excluir ejemplos con errores.
        """
        clean_results = [r for r in self.results if not r.get('error', False)] if filter_errors else self.results

        if not clean_results:
            print("No hay resultados válidos para crear dataset")
            return None

        ragas_data = {
            'question': [r['question'] for r in clean_results],
            'answer': [r['answer'] for r in clean_results],
            'contexts': [r['contexts'] for r in clean_results],
            'ground_truth': [r['ground_truth'] for r in clean_results]
        }

        dataset = Dataset.from_dict(ragas_data)
        print(f"📊 Dataset RAGAS creado: {len(dataset)} ejemplos")
        return dataset

    def get_summary(self):
        """Resumen del progreso."""
        if not self.results:
            return "No hay resultados aún"

        total = len(self.results)
        errors = sum(1 for r in self.results if r.get('error', False))
        successful = total - errors
        models_used = {r.get('model_used', 'unknown'): 0 for r in self.results}
        for r in self.results:
            model = r.get('model_used', 'unknown')
            models_used[model] = models_used.get(model, 0) + 1

        return f"""
📊 RESUMEN DEL PROGRESO:
- Total procesado: {total}
- Exitosos: {successful}
- Con errores: {errors}
- Modelos usados: {dict(models_used)}
- Dataset original: {len(self.dataset)} ejemplos
- Progreso: {(total/len(self.dataset)*100):.1f}%
        """

In [3]:
from datasets import load_dataset, Dataset
import math
import pandas as pd

# Cargar dataset
qa = load_dataset("UKPLab/PeerQA", "qa")["test"]

# Función mejorada para validar el ground truth
def is_valid_text(val):
    # Verificar si es None
    if val is None:
        return False
    
    # Verificar si es NaN float
    if isinstance(val, float) and math.isnan(val):
        return False
    
    # Verificar si es pandas NaN
    if pd.isna(val):
        return False
    
    # Convertir a string y verificar
    val_str = str(val).strip().lower()
    
    # Verificar si es string vacío o variaciones de "nan"
    if val_str == "" or val_str in ["nan", "none", "null", "<na>", "n/a"]:
        return False
    
    return True

# Extraer ground truth con prioridad y validar
def get_ground_truth(example):
    for key in ["answer_free_form_augmented", "answer_free_form"]:
        val = example.get(key)
        if is_valid_text(val):
            return val
    return None

# Función para validar contextos
def is_valid_context(context):
    if not context:
        return False
    if isinstance(context, list):
        # Verificar que la lista no esté vacía y tenga elementos válidos
        return len(context) > 0 and all(is_valid_text(item) for item in context)
    return is_valid_text(context)

# Construir dataset filtrado con más debugging
examples = []
total_examples = len(qa)
filtered_stats = {
    "not_answerable": 0,
    "invalid_paper_id": 0,
    "no_ground_truth": 0,
    "invalid_context": 0,
    "valid": 0
}

print(f"Procesando {total_examples} ejemplos...")

for i, ex in enumerate(qa):
    # Debug cada 1000 ejemplos
    if i % 1000 == 0:
        print(f"Procesando ejemplo {i}/{total_examples}")
    
    # Verificar si es answerable
    if not ex["answerable"]:
        filtered_stats["not_answerable"] += 1
        continue

    # Verificar paper_id
    if not is_valid_text(ex["paper_id"]):
        filtered_stats["invalid_paper_id"] += 1
        continue
    
    # Obtener ground truth
    ground_truth = get_ground_truth(ex)
    if not ground_truth:
        filtered_stats["no_ground_truth"] += 1
        # Debug: imprimir algunos casos problemáticos
        if filtered_stats["no_ground_truth"] <= 5:
            print(f"Ejemplo sin ground_truth válido:")
            print(f"  answer_free_form_augmented: {repr(ex.get('answer_free_form_augmented'))}")
            print(f"  answer_free_form: {repr(ex.get('answer_free_form'))}")
        continue
    
    # Verificar contexto
    context = ex["answer_evidence_sent"] or ex["raw_answer_evidence"]
    if not is_valid_context(context):
        filtered_stats["invalid_context"] += 1
        continue
    
    # Si llegamos aquí, el ejemplo es válido
    filtered_stats["valid"] += 1
    examples.append({
        "question": ex["question"],
        "ground_truth": ground_truth,
        "contexts": context,
        "paper_id": ex["paper_id"]
    })

# Crear dataset limpio
qa_clean_dataset = Dataset.from_list(examples)

print(f"\nNúmero final de ejemplos válidos: {len(qa_clean_dataset)}")

Using the latest cached version of the dataset since UKPLab/PeerQA couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'qa' at C:\Users\danie\.cache\huggingface\datasets\UKPLab___peer_qa\qa\1.0.0\2b4c608f2e508bafc5d874167212597ed9d3351a9f107dfa83f628a4bdfbc337 (last modified on Sun Jul 20 11:03:10 2025).


Procesando 579 ejemplos...
Procesando ejemplo 0/579
Ejemplo sin ground_truth válido:
  answer_free_form_augmented: 'nan'
  answer_free_form: None
Ejemplo sin ground_truth válido:
  answer_free_form_augmented: 'nan'
  answer_free_form: None
Ejemplo sin ground_truth válido:
  answer_free_form_augmented: 'nan'
  answer_free_form: None
Ejemplo sin ground_truth válido:
  answer_free_form_augmented: 'nan'
  answer_free_form: None
Ejemplo sin ground_truth válido:
  answer_free_form_augmented: 'nan'
  answer_free_form: None

Número final de ejemplos válidos: 267


In [4]:
import os
import json
import time
import datetime
import torch
import numpy as np
import pandas as pd
from typing import List, Dict, Any, Optional, Tuple
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv
from docling.document_converter import DocumentConverter
from sentence_transformers import SentenceTransformer, CrossEncoder
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import groq
from openai import OpenAI
from datasets import Dataset, load_dataset
import nltk
from nltk.tokenize import sent_tokenize
from rank_bm25 import BM25Okapi
from collections import defaultdict
import re
import google.generativeai as genai

try:
    import spacy
except ImportError:
    spacy = None

# Descargar recursos NLTK
try:
    nltk.download('punkt', quiet=True)
except:
    pass

load_dotenv("./.env", override=True)

class OptimizedRAGPipeline:
    """
    RAG Optimizado:
    - Cross-encoder re-ranking
    - Búsqueda híbrida (Dense + BM25)
    - Context expansion
    - Query expansion
    - Prompt engineering avanzado
    """
    
    def __init__(self, 
                 qdrant_host="localhost", 
                 qdrant_port=6333,
                 collection_name="advanced_rag_docs",
                 device="auto"):
        
        print("Inicializando RAG Optimizado...")
        
        # Configurar dispositivo
        if device == "auto":
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device
        print(f"Dispositivo: {self.device}")
        
        # 1. Docling (mantenido)
        self.doc_converter = DocumentConverter()
        
        # 2. BGE-M3 embeddings (mantenido)
        print("Cargando BGE-M3...")
        self.embedding_model = SentenceTransformer(
            'BAAI/bge-m3',
            device=self.device,
        )
        self.embedding_dim = 1024
        
        # 3. NUEVO: Cross-encoder para re-ranking
        print("🎯 Cargando Cross-encoder...")
        try:
            self.cross_encoder = CrossEncoder(
                'cross-encoder/ms-marco-MiniLM-L-12-v2',
                device=self.device
            )
            print("Cross-encoder cargado")
        except Exception as e:
            print(f"Error cargando cross-encoder: {e}")
            self.cross_encoder = None
        
        # 4. Qdrant (mantenido)
        print("🗄️ Conectando Qdrant...")
        self.qdrant_client = QdrantClient(host=qdrant_host, port=qdrant_port)
        self.collection_name = collection_name
        self._setup_collection()
     
        # 5. NUEVO: BM25 Index (se construye dinámicamente)
        self.bm25_index = None
        self.bm25_docs = []
        self.bm25_metadata = []
        
        # 6. spaCy para chunks semánticos (mantenido)
        try:
            self.nlp = spacy.load("es_core_news_sm")
        except:
            print("Modelo español no encontrado, usando inglés")
            try:
                self.nlp = spacy.load("en_core_web_sm")
            except:
                print("Modelo inglés no encontrado, usando básico")
                self.nlp = spacy.blank("es")
        
        # 7. Clientes LLM (corregido)
        self.gemini_client = None
        gemini_key = os.getenv("GEMINI_API_KEY")
        if gemini_key:
            genai.configure(api_key=gemini_key)
            self.gemini_client = genai.GenerativeModel('gemini-2.0-flash')
        
        self.groq_client = None
        self.openrouter_client = None
        
        # Inicializar Groq si hay API key
        groq_key = os.getenv("GROQ_API_KEY")
        if groq_key:
            self.groq_client = groq.Groq(api_key=groq_key)
        
        # Inicializar OpenRouter si hay API key  
        openrouter_key = os.getenv("OPENROUTE_API_KEY")
        if openrouter_key:
            self.openrouter_client = OpenAI(
                base_url="https://openrouter.ai/api/v1",
                api_key=openrouter_key
            )
        
        print("RAG Optimizado inicializado!")
    
    def _setup_collection(self):
        """Crear colección Qdrant (mantenido)"""
        try:
            collections = self.qdrant_client.get_collections().collections
            if not any(c.name == self.collection_name for c in collections):
                self.qdrant_client.create_collection(
                    collection_name=self.collection_name,
                    vectors_config=VectorParams(size=self.embedding_dim, distance=Distance.COSINE)
                )
                print(f"Colección '{self.collection_name}' creada")
            else:
                print(f"Colección '{self.collection_name}' ya existe")
        except Exception as e:
            print(f"Error al configurar colección: {e}")

    def _build_bm25_index(self, force_rebuild=False):
        """NUEVO: Construir índice BM25 a partir de documentos en Qdrant"""
        if self.bm25_index is not None and not force_rebuild:
            return
            
        print("🔍 Construyendo índice BM25...")
        
        try:
            # Recuperar todos los documentos de Qdrant
            all_docs = []
            offset = None
            
            while True:
                results = self.qdrant_client.scroll(
                    collection_name=self.collection_name,
                    limit=1000,
                    offset=offset,
                    with_payload=True
                )
                
                points, next_offset = results
                if not points:
                    break
                    
                all_docs.extend(points)
                offset = next_offset
                
                if next_offset is None:
                    break
            
            print(f"Recuperados {len(all_docs)} documentos para BM25")
            
            # Preparar textos y metadatos
            self.bm25_docs = []
            self.bm25_metadata = []
            
            for doc in all_docs:
                text = doc.payload.get('text', '')
                if text.strip():
                    # Tokenizar para BM25
                    tokens = self._tokenize_for_bm25(text)
                    self.bm25_docs.append(tokens)
                    self.bm25_metadata.append({
                        'id': doc.id,
                        'doc_id': doc.payload.get('doc_id'),
                        'chunk_id': doc.payload.get('chunk_id'),
                        'text': text,
                        'tokens': doc.payload.get('tokens', len(tokens))
                    })
            
            # Construir índice BM25
            if self.bm25_docs:
                self.bm25_index = BM25Okapi(self.bm25_docs)
                print(f"Índice BM25 construido con {len(self.bm25_docs)} documentos")
            else:
                print("No se encontraron documentos para BM25")
                
        except Exception as e:
            print(f"Error construyendo índice BM25: {e}")
            self.bm25_index = None
    
    def _tokenize_for_bm25(self, text: str) -> List[str]:
        """NUEVO: Tokenización para BM25"""
        # Limpiar y tokenizar
        text = re.sub(r'[^\w\s]', ' ', text.lower())
        tokens = [token for token in text.split() if len(token) > 2]
        return tokens
    
    def _expand_query(self, query: str) -> List[str]:
        """NUEVO: Expandir query con sinónimos y variaciones"""
        queries = [query]
        
        # Variaciones básicas
        query_lower = query.lower()
        if query != query_lower:
            queries.append(query_lower)
        
        # Agregar variaciones sin stopwords
        stopwords = {'el', 'la', 'los', 'las', 'un', 'una', 'de', 'del', 'en', 'con', 'por', 'para', 'que', 'se', 'es', 'son', 'fue', 'fueron'}
        words = [w for w in query.split() if w.lower() not in stopwords]
        if len(words) > 1:
            queries.append(' '.join(words))
        
        # Variaciones con palabras clave
        if 'modelo' in query_lower:
            queries.append(query.replace('modelo', 'model').replace('Modelo', 'Model'))
        if 'datos' in query_lower:
            queries.append(query.replace('datos', 'data').replace('Datos', 'Data'))
        if 'precisión' in query_lower or 'accuracy' in query_lower:
            queries.append(query.replace('precisión', 'accuracy').replace('Precisión', 'Accuracy'))
        
        return queries[:3]  # Máximo 3 variaciones
    
    def _hybrid_search(self, query: str, top_k: int = 10) -> List[Dict]:
        """NUEVO: Búsqueda híbrida (Dense + BM25)"""
        results = []
        
        # 1. Búsqueda densa (semántica) - ORIGINAL
        try:
            query_emb = self.embedding_model.encode([query], normalize_embeddings=True)[0]
            dense_results = self.qdrant_client.search(
                collection_name=self.collection_name,
                query_vector=query_emb.tolist(),
                limit=min(top_k * 3, 100),
                with_payload=True
            )
            
            for result in dense_results:
                results.append({
                    'id': result.id,
                    'text': result.payload['text'],
                    'doc_id': result.payload['doc_id'],
                    'chunk_id': result.payload['chunk_id'],
                    'dense_score': result.score,
                    'bm25_score': 0.0,
                    'source': 'dense'
                })
        except Exception as e:
            print(f"⚠️ Error en búsqueda densa: {e}")
        
        # 2. Búsqueda BM25 (léxica) - NUEVO
        if self.bm25_index is None:
            self._build_bm25_index()
        
        if self.bm25_index is not None:
            try:
                # Expandir query
                expanded_queries = self._expand_query(query)
                bm25_scores_combined = defaultdict(float)
                
                for q in expanded_queries:
                    query_tokens = self._tokenize_for_bm25(q)
                    scores = self.bm25_index.get_scores(query_tokens)
                    
                    for i, score in enumerate(scores):
                        if score > 0:
                            bm25_scores_combined[i] += score / len(expanded_queries)
                
                # Agregar mejores resultados BM25
                for idx, score in sorted(bm25_scores_combined.items(), key=lambda x: x[1], reverse=True)[:top_k]:
                    if idx < len(self.bm25_metadata):
                        meta = self.bm25_metadata[idx]
                        # Verificar si ya existe (por dense search)
                        existing = next((r for r in results if r['id'] == meta['id']), None)
                        if existing:
                            existing['bm25_score'] = score
                            existing['source'] = 'hybrid'
                        else:
                            results.append({
                                'id': meta['id'],
                                'text': meta['text'],
                                'doc_id': meta['doc_id'],
                                'chunk_id': meta['chunk_id'],
                                'dense_score': 0.0,
                                'bm25_score': score,
                                'source': 'bm25'
                            })
            except Exception as e:
                print(f"⚠️ Error en búsqueda BM25: {e}")
        
        # 3. Combinar scores híbridos
        for result in results:
            # Normalizar scores (0-1)
            dense_norm = result['dense_score']  # Ya está normalizado por cosine
            bm25_norm = min(result['bm25_score'] / 10.0, 1.0) if result['bm25_score'] > 0 else 0.0
            
            # Score híbrido
            result['hybrid_score'] = (dense_norm * 0.6) + (bm25_norm * 0.4)
        
        # Ordenar por score híbrido
        results.sort(key=lambda x: x['hybrid_score'], reverse=True)
        return results[:top_k]
    
    def _cross_encoder_rerank(self, query: str, results: List[Dict], top_k: int = 5) -> List[Dict]:
        """NUEVO: Re-ranking con cross-encoder"""
        if not results or self.cross_encoder is None:
            return results[:top_k]
        
        try:
            # Preparar pares query-documento
            pairs = [[query, result['text']] for result in results]
            
            # Obtener scores del cross-encoder
            ce_scores = self.cross_encoder.predict(pairs)
            
            # Actualizar results con scores del cross-encoder
            for result, ce_score in zip(results, ce_scores):
                result['ce_score'] = float(ce_score)
                # Score final combina híbrido y cross-encoder
                result['final_score'] = (result['hybrid_score'] * 0.4) + (ce_score * 0.6)
            
            # Re-ordenar por score final
            results.sort(key=lambda x: x['final_score'], reverse=True)
            print(f"Cross-encoder reranking aplicado a {len(results)} resultados")
            
        except Exception as e:
            print(f"Error en cross-encoder: {e}")
            # Fallback: usar solo score híbrido
            for result in results:
                result['final_score'] = result['hybrid_score']
        
        return results[:top_k]
    
    def _expand_context(self, results: List[Dict], expansion_size: int = 1) -> List[Dict]:
        """NUEVO: Expandir contexto con chunks adyacentes"""
        expanded_results = []
        
        for result in results:
            doc_id = result['doc_id']
            chunk_id = result['chunk_id']
            
            # Buscar chunks adyacentes
            adjacent_chunks = []
            for offset in range(-expansion_size, expansion_size + 1):
                if offset == 0:
                    continue  # Skip el chunk actual
                
                target_chunk_id = chunk_id + offset
                try:
                    # Buscar chunk adyacente
                    adjacent_results = self.qdrant_client.scroll(
                        collection_name=self.collection_name,
                        scroll_filter={
                            "must": [
                                {"key": "doc_id", "match": {"value": doc_id}},
                                {"key": "chunk_id", "match": {"value": target_chunk_id}}
                            ]
                        },
                        limit=1,
                        with_payload=True
                    )
                    
                    if adjacent_results[0]:
                        adjacent_chunks.append((offset, adjacent_results[0][0].payload['text']))
                        
                except Exception as e:
                    continue
            
            # Construir texto expandido
            expanded_text = result['text']
            
            # Agregar chunks anteriores
            before_chunks = [text for offset, text in sorted(adjacent_chunks) if offset < 0]
            if before_chunks:
                expanded_text = " [...] " + " ".join(before_chunks) + " " + expanded_text
            
            # Agregar chunks posteriores  
            after_chunks = [text for offset, text in sorted(adjacent_chunks) if offset > 0]
            if after_chunks:
                expanded_text = expanded_text + " " + " ".join(after_chunks) + " [...]"
            
            # Crear resultado expandido
            expanded_result = result.copy()
            expanded_result['text'] = expanded_text
            expanded_result['expanded'] = len(adjacent_chunks) > 0
            expanded_results.append(expanded_result)
        
        return expanded_results
    
    def optimized_search(self, query: str, top_k: int = 5, expand_context: bool = True) -> List[Dict]:
        """NUEVO: Búsqueda optimizada completa"""
        print(f"🔍 Búsqueda optimizada: {query[:50]}...")
        
        # 1. Búsqueda híbrida
        hybrid_results = self._hybrid_search(query, top_k * 3)
        print(f"Resultados híbridos: {len(hybrid_results)}")
        
        # 2. Re-ranking con cross-encoder
        reranked_results = self._cross_encoder_rerank(query, hybrid_results, top_k * 2)
        print(f"Después de re-ranking: {len(reranked_results)}")
        
        # 3. Expansión de contexto
        if expand_context:
            expanded_results = self._expand_context(reranked_results, expansion_size=0)
            print(f"Contextos expandidos: {sum(1 for r in expanded_results if r.get('expanded', False))}")
        else:
            expanded_results = reranked_results
        
        return expanded_results[:top_k]
    
    def advanced_generate_answer(self, query: str, contexts: List[str], model: str = "groq") -> str:
        """NUEVO: Generación avanzada con mejor prompt engineering"""
        if not contexts:
            return "No se encontró información relevante en los documentos indexados."
        
        # Preparar contextos numerados y mejorados
        context_text = ""
        for i, ctx in enumerate(contexts[:5], 1):  # Máximo 5 contextos
            context_text += f"\n[CONTEXTO {i}]\n{ctx}\n"
        
        # Prompt mejorado con técnicas avanzadas
        prompt = f"""Eres un asistente experto en análisis de documentos académicos y técnicos. Tu tarea es responder preguntas basándote en los contextos proporcionados.

INSTRUCCIONES:
1. Analiza cuidadosamente todos los contextos proporcionados
2. Si la información está presente, proporciona una respuesta completa y detallada
3. Si la información está parcialmente presente, indica qué partes puedes responder y cuáles no
4. no utilices un lenguaje maquinal o robotizado, el tono de las respuestas debe ser natural y profesional como si fueras un asistente
5. Si necesitas hacer inferencias lógicas basadas en la información disponible, hazlo explícitamente evitando alucinaciones
6. Solo indica "no tengo suficiente información" si realmente no hay nada relevante en los contextos

CONTEXTOS DISPONIBLES:{context_text}

PREGUNTA: {query}

ANÁLISIS Y RESPUESTA:"""
        
        try:
            if model == "groq" and self.groq_client:
                response = self.groq_client.chat.completions.create(
                    model="llama3-70b-8192",
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.5,  # Ligeramente más alta para flexibilidad
                    max_tokens=1024   # Más tokens para respuestas completas
                )
                return response.choices[0].message.content
                
            elif model == "openrouter" and self.openrouter_client:
                response = self.openrouter_client.chat.completions.create(
                    model="qwen/qwen3-32b",
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.5,
                    max_tokens=1024
                )
                return response.choices[0].message.content
            elif model == "gemini" and self.gemini_client:  # AGREGAR ESTO
                response = self.gemini_client.generate_content(prompt)
                return response.text
            else:
                return f"Modelo '{model}' no disponible. Verifica las API keys."
                
        except Exception as e:
            return f"Error generando respuesta: {str(e)}"
    
    def query_rag_optimized(self, question: str, model: str = "groq", top_k: int = 5) -> Dict:
        print(f"Query optimizada: {question[:50]}...")
        
        # Búsqueda optimizada
        results = self.optimized_search(question, top_k, expand_context=True)
        contexts = [r['text'] for r in results]
        
        if not contexts:
            answer = "No se encontraron documentos relevantes para responder la pregunta."
        else:
            # Generación avanzada
            answer = self.advanced_generate_answer(question, contexts, model)
        
        return {
            'question': question,
            'answer': answer,
            'contexts': contexts,
            'sources': [r['doc_id'] for r in results],
            'search_details': {
                'total_results': len(results),
                'expanded_contexts': sum(1 for r in results if r.get('expanded', False)),
                'avg_hybrid_score': np.mean([r['hybrid_score'] for r in results]) if results else 0,
                'avg_final_score': np.mean([r.get('final_score', 0) for r in results]) if results else 0
            },
            'model': model,
            'timestamp': datetime.now().isoformat()
        }

    
    def document_exists(self, doc_id: str) -> bool:
        """Verificar si el documento ya está indexado (mantenido)"""
        try:
            results = self.qdrant_client.scroll(
                collection_name=self.collection_name,
                scroll_filter={"must": [{"key": "doc_id", "match": {"value": doc_id}}]},
                limit=1
            )
            return len(results[0]) > 0
        except:
            return False
    
    def extract_with_docling(self, file_path: str) -> str:
        """Extraer texto con Docling (mantenido)"""
        try:
            result = self.doc_converter.convert(file_path)
            
            if hasattr(result.document, 'export_to_markdown'):
                text = result.document.export_to_markdown()
            elif hasattr(result.document, 'export_to_text'):
                text = result.document.export_to_text()
            elif hasattr(result.document, 'body'):
                text = str(result.document.body)
            else:
                text = str(result.document)
                
            print(f"Texto extraído: {len(text)} caracteres")
            return text
            
        except Exception as e:
            print(f"Error Docling: {e}")
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    return f.read()
            except:
                return ""
    
    def create_semantic_chunks(self, text: str, max_tokens: int = 400) -> List[Dict]:
        """Crear chunks semánticos (mantenido)"""
        if not text.strip():
            return []
            
        try:
            doc = self.nlp(text)
            sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]
        except:
            try:
                sentences = sent_tokenize(text)
            except:
                sentences = [s.strip() for s in text.split('.') if s.strip()]
        
        if not sentences:
            sentences = [p.strip() for p in text.split('\n\n') if p.strip()]
        
        chunks = []
        current_chunk = []
        current_tokens = 0
        
        for sentence in sentences:
            tokens = len(sentence.split())
            
            if current_tokens + tokens > max_tokens and current_chunk:
                chunks.append({
                    'text': ' '.join(current_chunk),
                    'tokens': current_tokens,
                    'id': len(chunks)
                })
                current_chunk = [sentence]
                current_tokens = tokens
            else:
                current_chunk.append(sentence)
                current_tokens += tokens
        
        if current_chunk:
            chunks.append({
                'text': ' '.join(current_chunk),
                'tokens': current_tokens,
                'id': len(chunks)
            })
        
        return chunks
    
    def index_document(self, file_path: str, doc_id: str = None) -> bool:
        print(f"Indexando: {file_path}")
        
        if not os.path.exists(file_path):
            print(f"Archivo no encontrado: {file_path}")
            return False
        
        doc_id = doc_id or Path(file_path).stem
        if self.document_exists(doc_id):
            print(f"Documento '{doc_id}' ya existe - saltando indexación")
            return True
        
        text = self.extract_with_docling(file_path)
        if not text or len(text.strip()) < 50:
            print(f"No se pudo extraer texto válido del archivo")
            return False
        
        chunks = self.create_semantic_chunks(text)
        if not chunks:
            print(f"No se pudieron crear chunks")
            return False
            
        print(f"Chunks creados: {len(chunks)}")
        
        texts = [chunk['text'] for chunk in chunks]
        try:
            embeddings = self.embedding_model.encode(
                texts, 
                batch_size=8,
                normalize_embeddings=True,
                show_progress_bar=True
            )
        except Exception as e:
            print(f"Error generando embeddings: {e}")
            return False
        
        points = []
        for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
            points.append(PointStruct(
                id=hash(f"{doc_id}_{i}") & 0x7FFFFFFF,
                vector=embedding.tolist(),
                payload={
                    'doc_id': doc_id,
                    'chunk_id': i,
                    'text': chunk['text'],
                    'tokens': chunk['tokens'],
                    'file_path': file_path,
                    'timestamp': datetime.now().isoformat()
                }
            ))
        
        try:
            self.qdrant_client.upsert(
                collection_name=self.collection_name,
                points=points
            )
            print(f"Indexado: {len(points)} chunks")
            return True
        except Exception as e:
            print(f"Error insertando en Qdrant: {e}")
            return False
    
    def setup_peerqa_retriever(self, papers_dataset=None):
        """Configura el retriever para PeerQA (mantenido)"""
        print("🚀 Configurando retriever para PeerQA...")
        
        if papers_dataset is None:
            papers_dataset = load_dataset("UKPLab/PeerQA", "papers")["test"]
        
        papers_dict = {}
        for paper in papers_dataset:
            paper_id = paper['paper_id']
            content = paper['content']
            if paper_id not in papers_dict:
                papers_dict[paper_id] = []
            if content.strip():
                papers_dict[paper_id].append(content)
        
        indexed = 0
        failed = 0
        
        for paper_id, contents in papers_dict.items():
            try:
                if self.document_exists(paper_id):
                    print(f"Documento '{paper_id}' ya indexado")
                    indexed += 1
                    continue
                
                text = " ".join(contents)
                if not text.strip() or len(text) < 50:
                    print(f"Texto vacío o muy corto para paper_id: {paper_id}")
                    failed += 1
                    continue
                
                chunks = self.create_semantic_chunks(text)
                if not chunks:
                    print(f"No se pudieron crear chunks para {paper_id}")
                    failed += 1
                    continue
                
                texts = [chunk['text'] for chunk in chunks]
                embeddings = self.embedding_model.encode(
                    texts,
                    batch_size=8,
                    normalize_embeddings=True,
                    show_progress_bar=True
                )
                
                points = [
                    PointStruct(
                        id=hash(f"{paper_id}_{i}") & 0x7FFFFFFF,
                        vector=embedding.tolist(),
                        payload={
                            'doc_id': paper_id,
                            'chunk_id': i,
                            'text': chunk['text'],
                            'tokens': chunk['tokens'],
                            'timestamp': datetime.now().isoformat()
                        }
                    ) for i, (chunk, embedding) in enumerate(zip(chunks, embeddings))
                ]
                
                self.qdrant_client.upsert(
                    collection_name=self.collection_name,
                    points=points
                )
                print(f"Indexado: {paper_id} ({len(points)} chunks)")
                indexed += 1
                
            except Exception as e:
                print(f"Error indexando {paper_id}: {str(e)}")
                failed += 1
        
        print(f"Indexación completada: {indexed} exitosos, {failed} fallidos")
        
        # IMPORTANTE: Rebuild BM25 index después de indexar nuevos documentos
        if indexed > 0:
            print("Reconstruyendo índice BM25...")
            self._build_bm25_index(force_rebuild=True)
        
        return indexed, failed

    def get_collection_info(self) -> Dict:
        """Información de la colección (mantenido)"""
        try:
            info = self.qdrant_client.get_collection(self.collection_name)
            return {
                'name': self.collection_name,
                'points_count': info.points_count,
                'vectors_count': info.vectors_count,
                'indexed': info.indexed_vectors_count,
                'bm25_ready': self.bm25_index is not None
            }
        except Exception as e:
            return {'error': str(e)}

    # MÉTODOS DE COMPATIBILIDAD (wrappers para métodos originales)
    
    def agent_search(self, query: str, top_k: int = 5) -> List[Dict]:
        """Wrapper para compatibilidad - usa búsqueda optimizada"""
        results = self.optimized_search(query, top_k, expand_context=False)
        
        # Convertir formato para compatibilidad
        compatible_results = []
        for r in results:
            compatible_results.append({
                'text': r['text'],
                'doc_id': r['doc_id'], 
                'score': r.get('final_score', r['hybrid_score']),
                'chunk_id': r['chunk_id']
            })
        
        return compatible_results
    
    def generate_answer(self, query: str, contexts: List[str], model: str = "groq") -> str:
        """Wrapper para compatibilidad - usa generación avanzada"""
        return self.advanced_generate_answer(query, contexts, model)
    
    def query_rag(self, question: str, model: str = "groq", top_k: int = 5) -> Dict:
        """Wrapper para compatibilidad - usa query optimizada"""
        return self.query_rag_optimized(question, model, top_k)

In [5]:
rag = OptimizedRAGPipeline()

Inicializando RAG Optimizado...
Dispositivo: cuda
Cargando BGE-M3...
🎯 Cargando Cross-encoder...
Cross-encoder cargado
🗄️ Conectando Qdrant...
Colección 'advanced_rag_docs' ya existe
RAG Optimizado inicializado!


In [11]:
import os
from IPython.display import Markdown, display

# 1. Inicializar el pipeline RAG
rag = OptimizedRAGPipeline(
    qdrant_host="localhost", 
    qdrant_port=6333,
    collection_name="advanced_rag_docs",
    device="auto"
)

# 2. Indexar un archivo PDF
pdf_path = r'C:\Users\danie\Downloads\prueba\data\1-s2.0-S0378517324005799-main.pdf'
if not os.path.exists(pdf_path):
    print(f"Error: El archivo {pdf_path} no existe")
else:
    success = rag.index_document(pdf_path, doc_id="my_pdf_document")
    if success:
        print(f"PDF {pdf_path} indexado correctamente")
    else:
        print(f"Error al indexar {pdf_path}")

# 3. Consulta optimizada
query = "¿Qué beneficios ofrece el uso de nanopartículas en comparación con los medicamentos tradicionales en el tratamiento del cáncer?"
result = rag.query_rag_optimized(query, model="groq")

# 4. Mostrar resultados
display(Markdown(f"\nPregunta: {result['question']}"))
print(f"Respuesta: {result['answer']}")

Inicializando RAG Optimizado...
Dispositivo: cuda
Cargando BGE-M3...
🎯 Cargando Cross-encoder...
Cross-encoder cargado
🗄️ Conectando Qdrant...
Colección 'advanced_rag_docs' ya existe
RAG Optimizado inicializado!
Indexando: C:\Users\danie\Downloads\prueba\data\1-s2.0-S0378517324005799-main.pdf
Documento 'my_pdf_document' ya existe - saltando indexación
PDF C:\Users\danie\Downloads\prueba\data\1-s2.0-S0378517324005799-main.pdf indexado correctamente
Query optimizada: ¿Qué beneficios ofrece el uso de nanopartículas en...
🔍 Búsqueda optimizada: ¿Qué beneficios ofrece el uso de nanopartículas en...
🔍 Construyendo índice BM25...
Recuperados 1513 documentos para BM25
Índice BM25 construido con 1513 documentos
Resultados híbridos: 15
Cross-encoder reranking aplicado a 15 resultados
Después de re-ranking: 10
Contextos expandidos: 0



Pregunta: ¿Qué beneficios ofrece el uso de nanopartículas en comparación con los medicamentos tradicionales en el tratamiento del cáncer?

Respuesta: Basándome en los contextos proporcionados, puedo responder a la pregunta de la siguiente manera:

El uso de nanopartículas en el tratamiento del cáncer ofrece varios beneficios en comparación con los medicamentos tradicionales. Según los contextos, estos beneficios incluyen:

1. **Mayor eficacia**: Las nanopartículas permiten una entrega más precisa y eficiente de los medicamentos a las células objetivo, lo que conduce a una mayor eficacia en el tratamiento del cáncer.

2. **Reducción de efectos secundarios**: Las nanopartículas pueden reducir la toxicidad y los efectos secundarios de los medicamentos, ya que solo se dirigen a las células objetivo y no a tejidos sanos.

3. **Mejora de la adherencia del paciente**: Debido a la reducción de efectos secundarios, los pacientes son más propensos a adherirse a la terapia, lo que conduce a mejores resultados en el tratamiento.

4. **Acceso a moléculas hidrofóbicas**: Las nanopartículas pueden transportar moléculas hidrofóbicas que 

In [ ]:
rag_pipeline = OptimizedRAGPipeline()

# Crear generador
generator = AdvancedRAGEvaluationGenerator(
    rag_pipeline=rag_pipeline,
    clean_dataset=qa_clean_dataset,
    papers_dir=None,  
    batch_size=5,
    delay=30
)

# Indexar documentos (redundante, pero asegura que todos los papers estén indexados)
generator.index_documents()

# Generar respuestas (prueba con 30 ejemplos)
generator.generate_responses_batch(model_type='openrouter', max_samples=10)

# Ver resumen
print(generator.get_summary())

# Crear dataset RAGAS
ragas_dataset = generator.get_ragas_dataset()

Inicializando RAG Optimizado...
Dispositivo: cuda
Cargando BGE-M3...
🎯 Cargando Cross-encoder...
Cross-encoder cargado
🗄️ Conectando Qdrant...
Colección 'advanced_rag_docs' ya existe


Using the latest cached version of the dataset since UKPLab/PeerQA couldn't be found on the Hugging Face Hub


RAG Optimizado inicializado!
✅ Inicializado AdvancedRAGEvaluationGenerator con 267 ejemplos
Indexando documentos...


Found the latest cached dataset configuration 'papers' at C:\Users\danie\.cache\huggingface\datasets\UKPLab___peer_qa\papers\1.0.0\2b4c608f2e508bafc5d874167212597ed9d3351a9f107dfa83f628a4bdfbc337 (last modified on Tue Jul 15 20:49:57 2025).


🚀 Configurando retriever para PeerQA...
Documento 'nlpeer/F1000-22/10-72' ya indexado
Documento 'nlpeer/F1000-22/10-170' ya indexado
Documento 'nlpeer/F1000-22/10-838' ya indexado
Documento 'nlpeer/F1000-22/10-637' ya indexado
Documento 'nlpeer/F1000-22/11-404' ya indexado
Documento 'nlpeer/F1000-22/10-654' ya indexado
Documento 'nlpeer/F1000-22/10-890' ya indexado
Documento 'nlpeer/F1000-22/11-222' ya indexado
Documento 'nlpeer/F1000-22/11-195' ya indexado
Documento 'nlpeer/F1000-22/11-9' ya indexado
Documento 'nlpeer/PeerRead-CONLL2016/166' ya indexado
Documento 'nlpeer/PeerRead-CONLL2016/142' ya indexado
Documento 'nlpeer/PeerRead-CONLL2016/129' ya indexado
Documento 'nlpeer/PeerRead-CONLL2016/12' ya indexado
Documento 'nlpeer/PeerRead-CONLL2016/13' ya indexado
Documento 'nlpeer/COLING2020/1405' ya indexado
Documento 'nlpeer/COLING2020/939' ya indexado
Documento 'nlpeer/COLING2020/1265' ya indexado
Documento 'nlpeer/COLING2020/1570' ya indexado
Documento 'nlpeer/COLING2020/1775' ya 